# Import các thư viện cần thiết


In [5]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
# Hiển thị 6 chữ số thập phân
pd.set_option('display.float_format', '{:.6f}'.format)

# 1. Tiền xử lý dữ liệu thống kê từ video


Đọc dữ liệu thống kê video thành DataFrame.


In [6]:
file_path = "data/final_raw_videos.csv"

In [2]:
# numeric_columns = ["authorStats.diggCount", "authorStats.followerCount", "authorStats.followingCount",
#                    "authorStats.friendCount", "authorStats.heart", "authorStats.heartCount",
#                    "authorStats.videoCount", "stats.collectCount", "stats.commentCount", "stats.playCount",
#                    "stats.shareCount", "statsV2.collectCount", "statsV2.commentCount", "statsV2.diggCount",
#                    "statsV2.playCount", "statsV2.repostCount", "statsV2.shareCount"
#                    ]

# bool_columns = ['author.openFavorite', 'author.verified', 'isAd', 'music.isCopyrighted',
#                 'music.original', 'textTranslatable', 'video.claInfo.enableAutoCaption',
#                 'video.claInfo.hasOriginalAudio']

In [3]:
# # Lấy tất cả các cột trong file
# sample_df = pd.read_csv(file_path, nrows=5)  # Đọc 5 dòng đầu để lấy tên cột
# all_columns = sample_df.columns.tolist()

# # Định nghĩa dtype
# dtype_dict = {
#     col: "float64" if col in numeric_columns else "str" for col in all_columns}

# video_info_df = pd.read_csv(file_path, low_memory=False, dtype=dtype_dict)
# video_info_df.info()

In [7]:
# Define only string columns
string_cols = ["author.id", "id", "music.id"]

# Create dtype mapping for string columns
dtype_schema = {col: "object" for col in string_cols}

# Read CSV with specified dtypes only for string columns
video_info_df = pd.read_csv(
    file_path, low_memory=False, dtype=dtype_schema)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Columns: 174 entries, stats.diggCount to duetInfo.duetFromId
dtypes: float64(87), int64(1), object(86)
memory usage: 94.6+ MB


Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%


In [8]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.tail()

item_control.can_repost   0.003705
itemCommentStatus         0.003705
isAd                      0.003705
id                        0.003705
stats.diggCount           0.000000
dtype: float64

In [9]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [10]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

video.claInfo.originalLanguageInfo.languageID     :  30.39%
video.claInfo.originalLanguageInfo.languageCode   :  30.39%
video.claInfo.captionsType                        :  30.39%
video.claInfo.originalLanguageInfo.canTranslateRealTimeNoCheck:  30.39%
video.claInfo.originalLanguageInfo.language       :  30.39%


In [ ]:
# video_info_df.info()

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [11]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [12]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Chuyển timestamp sang datetime và chuyển sang giờ VN.


In [13]:
# Chuyển timestamp sang dạng datetime UTC
video_info_df["createTime"] = pd.to_datetime(
    video_info_df["createTime"], unit="s", utc=True)
video_info_df["collectTime"] = pd.to_datetime(
    video_info_df["collectTime"], unit="s", utc=True)

# Chuyển sang múi giờ Việt Nam (UTC+7)
video_info_df["createTime"] = video_info_df["createTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")
video_info_df["collectTime"] = video_info_df["collectTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")

In [ ]:
video_info_df['hour'] = video_info_df['createTime'].dt.hour
px.histogram(video_info_df, x='hour', nbins=24)

Drop các cột chỉ có 1 giá trị phân biệt.


In [13]:
# Extract text (object) columns
# text_df = video_info_df.select_dtypes('object')
# unique_counts = text_df.nunique()
unique_counts = video_info_df.nunique()
one_value_cols = list(unique_counts[unique_counts == 1].index)
print(f"One value columns: {one_value_cols}")
video_info_df = video_info_df.drop(columns=one_value_cols)
video_info_df.columns

One value columns: ['author.commentSetting', 'author.ftc', 'author.isADVirtual', 'author.isEmbedBanned', 'author.privateAccount', 'author.relation', 'author.secret', 'authorStats.friendCount', 'collected', 'digged', 'duetDisplay', 'forFriend', 'item_control.can_repost', 'music.private', 'officalItem', 'originalItem', 'privateItem', 'secret', 'shareEnabled', 'statsV2.repostCount', 'stitchDisplay', 'video.claInfo.captionsType', 'video.format', 'duetEnabled', 'stitchEnabled']


Index(['CategoryType', 'author.downloadSetting', 'author.duetSetting',
       'author.id', 'author.nickname', 'author.openFavorite', 'author.secUid',
       'author.signature', 'author.stitchSetting', 'author.uniqueId',
       'author.verified', 'authorStats.diggCount', 'authorStats.followerCount',
       'authorStats.followingCount', 'authorStats.heart',
       'authorStats.heartCount', 'authorStats.videoCount', 'createTime',
       'desc', 'diversificationId', 'id', 'isAd', 'itemCommentStatus',
       'music.authorName', 'music.duration', 'music.id', 'music.isCopyrighted',
       'music.original', 'music.title', 'statsV2.collectCount',
       'statsV2.commentCount', 'statsV2.diggCount', 'statsV2.playCount',
       'statsV2.shareCount', 'textLanguage', 'textTranslatable',
       'video.VQScore', 'video.bitrate', 'video.claInfo.enableAutoCaption',
       'video.claInfo.hasOriginalAudio', 'video.codecType', 'video.definition',
       'video.duration', 'video.encodedType', 'video.height'

Chuyển các cột true/false về dtype boolean.


In [14]:
# Select columns where all values are 'True' or 'False' (as strings)
true_false_str_cols = video_info_df.columns[
    video_info_df.apply(lambda col: col.astype(
        str).isin(["True", "False"]).any())
]
print(true_false_str_cols)
video_info_df[true_false_str_cols] = video_info_df[true_false_str_cols].astype(
    'bool')
# video_info_df.info()

Index(['author.openFavorite', 'author.verified', 'isAd', 'music.isCopyrighted',
       'music.original', 'textTranslatable', 'video.claInfo.enableAutoCaption',
       'video.claInfo.hasOriginalAudio'],
      dtype='object')


Drop cột trùng lắp hoặc thông tin không hữu ích.


In [15]:
cols_to_delete = ['CategoryType',
                  'author.secUid',
                  #  'diversificationId',
                  'itemCommentStatus',
                  #   'video.bitrate', 'video.codecType', 'video.ratio',
                  "video.id",  # == "id"
                  'video.definition',  # == "video.ratio"
                  'video.videoID', ]

video_info_df = video_info_df.drop(columns=cols_to_delete)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Data columns (total 47 columns):
 #   Column                           Non-Null Count  Dtype                           
---  ------                           --------------  -----                           
 0   author.downloadSetting           70996 non-null  float64                         
 1   author.duetSetting               70996 non-null  float64                         
 2   author.id                        70996 non-null  object                          
 3   author.nickname                  70996 non-null  object                          
 4   author.openFavorite              71260 non-null  bool                            
 5   author.signature                 70140 non-null  object                          
 6   author.stitchSetting             70996 non-null  float64                         
 7   author.uniqueId                  70996 non-null  object                          
 8   author.verified 

Drop các dòng có thiếu hơn 80% số cột.


In [16]:
video_info_df["missing_count"] = video_info_df.isnull().sum(axis=1)
# Xóa các dòng có hơn 50% giá trị bị thiếu
threshold = video_info_df.shape[1] * 0.8

# Lọc bỏ các dòng có số lượng giá trị thiếu vượt quá ngưỡng
video_info_df = video_info_df[video_info_df["missing_count"] <= threshold].drop(
    columns=["missing_count"]).reset_index(drop=True)
video_info_df.shape

(70996, 47)

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.


In [17]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # # Remove emojis
        # description = description.encode('ascii', 'ignore').decode('ascii')
        # Chuyển tiếng Việt có dấu thành không dấu
        description = unidecode(description)

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 70996/70996 [00:10<00:00, 6456.79it/s]


In [18]:
video_info_df["hashtags"] = video_info_df["hashtags"].apply(
    lambda x: x.split(",") if x else [])

# 2. Video score (weekly)


In [19]:
video_info_df['year'] = video_info_df['createTime'].dt.isocalendar().year
video_info_df['week'] = video_info_df['createTime'].dt.isocalendar().week
video_info_df['engagement_rate'] = (video_info_df['statsV2.collectCount'] + video_info_df['statsV2.commentCount'] +
                                    video_info_df['statsV2.shareCount'] + video_info_df['statsV2.diggCount']) / video_info_df['statsV2.playCount']

In [20]:
stats_cols = ['statsV2.playCount',
              'statsV2.diggCount',
              'statsV2.commentCount',
              'statsV2.shareCount',
              'statsV2.collectCount',
              'engagement_rate'
              ]

In [21]:
def compute_weekly_score_and_rank(df):
    """
    Tính điểm tổng hợp theo tuần và xếp hạng video theo từng tuần, 
    đồng thời thêm xếp hạng riêng cho từng chỉ số.
    """

    df_copy = df.copy()

    # Trọng số của từng chỉ số
    columns_weights = {
        'statsV2.playCount': 0.40,
        "engagement_rate": 0.1,
        "statsV2.shareCount": 0.15,
        "statsV2.commentCount": 0.10,
        "statsV2.diggCount": 0.25
    }

    # Danh sách các cột cần xếp hạng riêng
    rank_columns = list(columns_weights.keys()) + \
        ["statsV2.collectCount"]

    # Xử lý NaN & giá trị âm
    for col in columns_weights.keys():
        df_copy[col] = df_copy[col].fillna(0).clip(lower=0)

    # Tính điểm và xếp hạng trong từng tuần
    df_copy = df_copy.groupby(["year", "week"], group_keys=False).apply(
        lambda group: _compute_score_and_ranking_per_week(
            group, columns_weights, rank_columns)
    )

    return df_copy


def _compute_score_and_ranking_per_week(group, columns_weights, rank_columns):
    """
    Tính điểm và xếp hạng trong từng tuần riêng lẻ.
    """
    # Chuẩn hóa theo max trong tuần
    max_values = {col: group[col].max() or 1 for col in columns_weights.keys()}

    for col in columns_weights.keys():
        group[f"norm_{col}"] = group[col] / max_values[col]

    # Tính điểm tổng hợp
    group["weekly_score"] = sum(
        columns_weights[col] * group[f"norm_{col}"] * 100
        for col in columns_weights.keys()
    )

    # Xếp hạng tổng điểm theo tuần
    group["weekly_score_rank"] = group["weekly_score"].rank(
        method="dense", ascending=False).astype("Int64")

    # Xếp hạng riêng cho từng chỉ số trong tuần
    for col in rank_columns:
        group[f"{col}_rank"] = group[col].rank(
            method="dense", ascending=False).astype("Int64")

    # Xóa cột chuẩn hóa sau khi tính toán xong
    norm_columns = [f"norm_{col}" for col in columns_weights.keys()]
    group = group.drop(columns=norm_columns)
    return group


# Gọi hàm trên DataFrame video_info_df
# selected_df = video_info_df[['year', 'week',
#                              'author.uniqueId', 'id', ]+stats_cols]
selected_df = video_info_df
selected_df = compute_weekly_score_and_rank(selected_df)

# Hiển thị số nguyên đầy đủ
with pd.option_context('display.float_format', '{:.3f}'.format):
    display(selected_df[selected_df['year'] == 2025].sort_values(
        ['year', 'week', 'weekly_score_rank'], ascending=True).head(5))

,author.downloadSetting,author.duetSetting,author.id,author.nickname,author.openFavorite,author.signature,author.stitchSetting,author.uniqueId,author.verified,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.heart,authorStats.heartCount,authorStats.videoCount,createTime,desc,diversificationId,id,isAd,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,statsV2.collectCount,statsV2.commentCount,statsV2.diggCount,statsV2.playCount,statsV2.shareCount,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.duration,video.encodedType,video.height,video.ratio,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,hashtags,num_hashtags,year,week,engagement_rate,weekly_score,weekly_score_rank,statsV2.playCount_rank,engagement_rate_rank,statsV2.shareCount_rank,statsV2.commentCount_rank,statsV2.diggCount_rank,statsV2.collectCount_rank
64251,3.000,0.000,7074548497097344026,Panda Chân Lý,False,👉🏻KHÔNG NHẬN BOOKING QUÁN ĂN,0.000,hoanganhdeptrailam,False,9725.000,912000.000,52.000,38500000.000,38500000.000,458.000,2025-01-01 13:00:00+07:00,Ốc vòi voi @panda.review0chuyen #ancungtiktok ...,10041.000,7454790808231890194,False,Panda Chân Lý,303.000,7454791839204608785,False,True,nhạc nền - Panda Chân Lý,13864.000,12700.000,226300.000,5300000.000,106500.000,vi,True,69.680,1168347.000,True,False,h264,303.000,normal,1024.000,540p,normal,-15.000,1.000,576.000,2025-03-19 16:53:11+07:00,"[ancungtiktok, tiktokgiaitri, hoanganhpanda]",3,2025,1,0.068,50.432,1,10,108,1,1,4,13
15795,3.000,0.000,6869980717103842305,Tóc Vàng hoe,False,Đồ makeup và quần áo Vàng mặc trong clip ở đây...,0.000,tocvang.hoe,False,968.000,2400000.000,5.000,44800000.000,44800000.000,404.000,2025-01-03 16:58:22+07:00,Mukbang blindbox uống trà Lipton nhãn vàng cùn...,10041.000,7455626418827824402,True,Tóc Vàng hoe,108.000,7455626546133437201,False,True,nhạc nền - Tóc Vàng hoe,4259.000,412.000,71600.000,20400000.000,156.000,vi,True,74.500,1088006.000,True,True,h264,108.000,normal,1024.000,540p,normal,-15.100,1.000,576.000,2025-03-19 16:56:53+07:00,"[tocvanghoe, mukbang, lipton, blindbox, ancung...",6,2025,1,0.004,44.818,2,1,1129,435,56,32,69
23025,0.000,0.000,7387796761811059718,Hoàng Anh Vào Bếp,False,Forwork booking\n📩 nguyenhoanganh.banker@gmail...,0.000,hoanganhvaobep,False,269.000,25400.000,91.000,264700.000,264700.000,67.000,2024-12-31 18:05:08+07:00,Phần 15/100: Trứng Rim Mắm Nay là ngày cuối củ...,10040.000,7454530370634059015,False,"𝓜𝓾𝓼𝓲𝓬 𝓬𝓱𝓲𝓵𝓵,🎧",300.000,7072602716777859866,False,True,"nhạc nền - 𝓜𝓾𝓼𝓲𝓬 𝓬𝓱𝓲𝓵𝓵,🎧",81266.000,3266.000,209000.000,11900000.000,38600.000,vi,True,70.880,1762722.000,True,True,h264,53.000,normal,1024.000,540p,normal,-26.300,0.351,576.000,2025-03-19 16:53:10+07:00,"[nauancungtiktok, learnontiktok, ancungtiktok,...",9,2025,1,0.028,44.670,3,3,577,3,3,6,1
67179,3.000,0.000,7001315265619936258,Ớt review,False,Liên hệ công việc : 0392415851 Ms Loan\nCẩm na...,0.000,ot.revieww,False,21800.000,618300.000,353.000,11300000.000,11300000.000,530.000,2025-01-04 17:27:32+07:00,Ăn cùng người lạ THPT An Dương p60 #otreview #...,10085.000,7456005004818681106,False,文武贝,170.000,7048037465826871327,False,False,Happy Day,23379.000,2445.000,405800.000,7800000.000,9727.000,vi,True,67.620,1064233.000,True,True,h264,137.000,normal,1024.000,540p,normal,-10.500,1.000,576.000,2025-03-19 16:59:19+07:00,"[otreview, wezmedia, xuhuong, vtmgr, ancungtik...",6,2025,1,0.057,44.506,4,6,166,19,4,1,6
23697,0.000,0.000,6883373303420175362,Trà Đé,False,Phở bò\nTrà Đé - Tiệm Trà Giải Khát\nCó trên s...,0.000,trade.travatrasua,False,1353.000,116100.000,21.000,2900000.000,2900000.000,512.000,2025-01-03 11:52:35+07:00,ly siêu dài 😋 #learnontiktok #xuhuong #trasua ...,10042.000,7455547606568062216,False,Trà Đé,15.000,7455547645868641041,False,True,nhạc nền - Trà Đé,19625.000,948.000,252800.000,8700000.0

In [80]:
v_df = selected_df[(selected_df['year'] == 2025) & (
    selected_df['week'] == 3)].sort_values(ascending=False, by='weekly_score').head(20)
v_df[['id', 'weekly_score_rank']+stats_cols]

,id,weekly_score_rank,statsV2.playCount,statsV2.diggCount,statsV2.commentCount,statsV2.shareCount,statsV2.collectCount,engagement_rate
16945,7460097038827441429,1,13900000.000000,525400.000000,4505.000000,33600.000000,25695.000000,0.042388
52954,7459740355940945170,2,12700000.000000,316500.000000,6165.000000,12600.000000,12749.000000,0.027403
23646,7461466064090565909,3,10000000.000000,364500.000000,1686.000000,13400.000000,26710.000000,0.040630
6891,7459803513753734401,4,6600000.000000,246000.000000,2228.000000,30300.000000,8423.000000,0.043477
61214,7460065770173762834,5,5600000.000000,236000.000000,1452.000000,41800.000000,57248.000000,0.060089
47882,7460841802707373329,6,8200000.000000,169600.000000,2052.000000,22100.000000,38726.000000,0.028351
62491,7459752350022896903,7,12700000.000000,15200.000000,49.000000,818.000000,943.000000,0.001339
61213,7460441203784518919,8,3900000.000000,162600.000000,1689.000000,48500.000000,8157.000000,0.056653
44930,7459246331064896786,9,5200000.000000,110400.000000,4703.000000,16900.000000,20010.000000,0.029233
15129,7460000590224215304,10,7700000.000000,98400.000000,1677.000000,6087.000000,5694.000000,0.014527


In [75]:
px.box(selected_df, x='weekly_score')

In [ ]:
top_score_df = selected_df.groupby(['year', 'week'])['weekly_score'].quantile(
    0.95).reset_index(name='weekly_score_1%')

merged_df = selected_df.merge(top_score_df, on=['year', 'week'])

merged_df['is_top_1_percent'] = merged_df['weekly_score'] >= merged_df['weekly_score_1%']
count_top_1p = merged_df.groupby(['year', 'week'])[
    'is_top_1_percent'].sum().reset_index(name='count_top_1_percent')
count_top_1p
count_top_1p['count_top_1_percent'].sum()

np.int64(3584)

In [22]:
# selected_df.to_csv(
#     "notebooks/preprocess/cleaned_videos_with_weekly_rank.csv", index=False)

# 3. Lọc lại danh sách user (BỎ)


In [24]:
import pandas as pd

# Đọc dữ liệu
df = pd.read_csv('notebooks/preprocess/cleaned_videos_with_weekly_rank.csv')

# -------------------------------------------
# 1. Trung bình view/video per user
user_stats = df.groupby('author.id').agg(
    total_views=('statsV2.playCount', 'sum'),
    total_videos=('id', 'count')
)
user_stats['avg_views'] = user_stats['total_views'] / \
    user_stats['total_videos']
q20_avg_view = user_stats['avg_views'].quantile(0.2)

print(f"🔹 Ngưỡng lọc trung bình view/video (Q20): {q20_avg_view:.0f}")
valid_user_avgview = user_stats[user_stats['avg_views'] >= q20_avg_view].index

# -------------------------------------------
# 2. Follower
q10_follower = df['authorStats.followerCount'].quantile(0.1)
print(f"🔹 Ngưỡng lọc follower (Q10): {q10_follower:.0f}")
df_follower_filtered = df[df['authorStats.followerCount'] >= q10_follower]
valid_user_follower = df_follower_filtered['author.id'].unique()

# -------------------------------------------
# 3. Engagement rate trung bình theo user
user_er = df.groupby('author.id')['engagement_rate'].mean()
q10_er = user_er.quantile(0.1)
print(f"🔹 Ngưỡng lọc engagement rate trung bình (Q10): {q10_er:.4f}")
valid_user_er = user_er[user_er >= q10_er].index

# -------------------------------------------
# 4. Tỷ lệ video flop (< Q25 view)
flop_threshold = df['statsV2.playCount'].quantile(0.25)
print(
    f"🔹 Ngưỡng view thấp (Q25 - định nghĩa video flop): {flop_threshold:.0f}")


def flop_ratio(group):
    flop = (group['statsV2.playCount'] < flop_threshold).sum()
    return flop / len(group)


user_flop_ratio = df.groupby('author.id').apply(flop_ratio)
q80_flop_ratio = user_flop_ratio.quantile(0.8)
print(f"🔹 Ngưỡng tỷ lệ video flop (Q80): {q80_flop_ratio:.2f}")
valid_user_flop = user_flop_ratio[user_flop_ratio <= q80_flop_ratio].index

# -------------------------------------------
# Giao các điều kiện lọc
valid_users = set(valid_user_avgview) & set(
    valid_user_follower) & set(valid_user_er) & set(valid_user_flop)

print(f"\n✅ Số lượng user ban đầu: {df['author.id'].nunique()}")
print(
    f"✅ Số lượng user sau khi lọc: {len(valid_users)} ({len(valid_users)/df['author.id'].nunique()*100:.1f}%)")

# Lọc lại dataframe chính
df_filtered = df[df['author.id'].isin(valid_users)]
print(
    f"✅ Số video sau khi lọc: {len(df_filtered)} / {len(df)} ({len(df_filtered)/len(df)*100:.1f}%)")

🔹 Ngưỡng lọc trung bình view/video (Q20): 134980
🔹 Ngưỡng lọc follower (Q10): 48200
🔹 Ngưỡng lọc engagement rate trung bình (Q10): 0.0207
🔹 Ngưỡng view thấp (Q25 - định nghĩa video flop): 32800
🔹 Ngưỡng tỷ lệ video flop (Q80): 0.51

✅ Số lượng user ban đầu: 264
✅ Số lượng user sau khi lọc: 170 (64.4%)
✅ Số video sau khi lọc: 47141 / 70996 (66.4%)


# 4. Another weekly score (BỎ)


In [ ]:
import pandas as pd
import numpy as np


def compute_weekly_ranking(df):
    """
    Tính điểm xếp hạng video theo tuần dựa trên:
    - Lượt xem tuyệt đối
    - Views per 1000 followers
    - Engagement rate

    Trả về dataframe đã thêm: new_score, new_rank (theo từng tuần)
    """

    # Kiểm tra cột thời gian và tạo thông tin tuần
    df['createTime'] = pd.to_datetime(df['createTime'], utc=True)
    # Chuyển sang múi giờ Việt Nam (UTC+7)
    df["createTime"] = df["createTime"].dt.tz_convert(
        "Asia/Ho_Chi_Minh")
    df['year'] = df['createTime'].dt.isocalendar().year
    df['week'] = df['createTime'].dt.isocalendar().week

    # Tính các chỉ số cần thiết
    df['views_per_1000_followers'] = df['statsV2.playCount'] / \
        (df['authorStats.followerCount'] / 1000 + 1e-6)
    df['engagement_rate'] = (
        df['statsV2.diggCount'] +
        df['statsV2.commentCount'] +
        df['statsV2.shareCount'] +
        df['statsV2.collectCount']
    ) / (df['statsV2.playCount'] + 1e-6)

    # Hàm áp dụng cho từng nhóm tuần
    def rank_group(group):
        # Chuẩn hóa các chỉ số trong tuần
        group['norm_views'] = group['statsV2.playCount'] / \
            group['statsV2.playCount'].max()
        group['norm_vpf'] = group['views_per_1000_followers'] / \
            group['views_per_1000_followers'].max()
        group['norm_er'] = group['engagement_rate'] / \
            group['engagement_rate'].max()

        # Tính điểm tổng hợp (trọng số đều)
        group['new_score'] = (group['norm_views'] * 0.6 +
                              group['norm_vpf'] * 0.25 + group['norm_er'] * 0.15) * 100

        # Xếp hạng trong tuần
        group = group.sort_values(
            'new_score', ascending=False).reset_index(drop=True)
        group['new_rank'] = np.arange(1, len(group)+1)
        return group

    df_ranked = df.groupby(
        ['year', 'week'], group_keys=False).apply(rank_group)

    return df_ranked


df_ranked = compute_weekly_ranking(df)

In [29]:
# Bước 1: Lấy top 20 theo score cụ thể
import matplotlib.pyplot as plt


def get_top20(df, score_col, col_suffix):
    top_df = (df
              .sort_values(['year', 'week', score_col], ascending=[True, True, False])
              .groupby(['year', 'week'])
              .head(20)
              [['year', 'week', 'id']]  # chỉ cần ID video
              )
    top_df = top_df.rename(columns={'id': f'id_{col_suffix}'})
    return top_df


# Lấy top 20 theo điểm mới và cũ
top20_new = get_top20(df_ranked, 'new_score', 'new')
top20_old = get_top20(df_ranked, 'weekly_score', 'old')

# Bước 2: Gộp lại theo tuần
merged = pd.merge(
    top20_new,
    top20_old,
    how='outer',
    on=['year', 'week']
)

# Bước 3: So sánh tập ID theo tuần
results = []

for (y, w), group in merged.groupby(['year', 'week']):
    new_ids = set(group['id_new'].dropna())
    old_ids = set(group['id_old'].dropna())
    unique_to_new = new_ids - old_ids
    results.append({
        'year': y,
        'week': w,
        'num_new_only': len(unique_to_new),
        'pct_changed': len(unique_to_new) / 20 * 100
    })

df_diff = pd.DataFrame(results).sort_values(['year', 'week'])

# Tổng kết
total_unique_to_new = df_diff['num_new_only'].sum()
avg_diff_per_week = df_diff['num_new_only'].mean()

print(
    f"\n📊 Tổng số video chỉ có mặt trong top 20 của new_score (không có trong weekly_score): {total_unique_to_new}")
print(
    f"🔁 Trung bình thay đổi mỗi tuần: {avg_diff_per_week:.2f} video (~{avg_diff_per_week/20*100:.1f}%)")

# Vẽ biểu đồ nếu cần

# plt.figure(figsize=(10, 4))
# plt.plot(df_diff['week'], df_diff['num_new_only'], marker='o')
# plt.title("Số video khác biệt trong top 20 (new_score vs weekly_score) theo tuần")
# plt.xlabel("Tuần")
# plt.ylabel("Số video chỉ có trong top 20 (new_score)")
# plt.grid(True, linestyle='--', alpha=0.6)
# plt.tight_layout()
# plt.show()


📊 Tổng số video chỉ có mặt trong top 20 của new_score (không có trong weekly_score): 305
🔁 Trung bình thay đổi mỗi tuần: 4.36 video (~21.8%)
